In [72]:
import pandas as pd
import numpy as np
import os
import cv2
# from tensorflow.keras.utils import load_img
import matplotlib.pyplot as pt

In [43]:
# input_path = []
# label = []
# for folder_name in os.listdir('PetImages'):
#     for folder_path in os.listdir('PetImages/'+folder_name):
#         if folder_name == 'Cat':
#             if folder_path.endswith('.jpg'):
#                 label.append('1')
#         else:
#             if folder_path.endswith('.jpg'):
#                 label.append('1')
#         input_path.append(os.path.join('PetImages', folder_name, folder_path))

In [68]:
# df = pd.DataFrame()
# df['images'] = input_path
# df['label'] = label
# df = df.sample(frac=1).reset_index(drop=True)
# df.head(5)

In [86]:
maindir = 'PetImages/'
categories = ['Cat', 'Dog']

training_list = []

for cat in categories:
    path = os.path.join(maindir, cat)
    cat_num = categories.index(cat)
    for img_file in os.listdir(path):
        img_arr = cv2.imread(os.path.join(path, img_file))
        training_list.append([img_arr, cat_num])

In [87]:
import random

random.shuffle(training_list) 

In [95]:
X = []
Y = []

for feat, label in training_list:
    X.append(feat)
    Y.append(label)

X = np.array(X).reshape(-1, 155, 155, 1)

In [96]:
# from sklearn.model_selection import train_test_split

# xtrain, ytrain, xtest, ytest = train_test_split(X, Y, test_size = 0.2, random_state = 10)

ValueError: Found input variables with inconsistent numbers of samples: [30006, 10002]

In [46]:
# from sklearn.model_selection import train_test_split
# train, test = train_test_split(df, test_size = 0.2, random_state = 0)

# from keras.preprocessing.image import ImageDataGenerator
# training_generator = ImageDataGenerator(rescale = 1./255)

# validation_generator = ImageDataGenerator(rescale = 1./255)
# training_iterator = training_generator.flow_from_dataframe(
#     train, 
#     x_col = 'images',
#     y_col = 'label',
#     target_size = (155,155),
#     batch_size=1024,
#     class_mode = 'binary'
# )

# validation_iterator = validation_generator.flow_from_dataframe(
#    test,
#    x_col = 'images',
#    y_col = 'label',
#    target_size=(155,155),
#    batch_size = 1024,
#    class_mode = 'binary'

# )

Found 8001 validated image filenames belonging to 2 classes.
Found 2001 validated image filenames belonging to 2 classes.


In [92]:
import keras
from keras import Sequential
from keras.layers import Conv2D, Flatten, MaxPool2D, Dense

model = Sequential()
model.add(Conv2D(16, (3,3), activation = 'relu', input_shape = (155,155,1)))
model.add(MaxPool2D(2,2))
model.add(Conv2D(32, (3,3), activation = 'relu'))
model.add(MaxPool2D(2,2))
model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(MaxPool2D(2,2))
model.add(Conv2D(128, (3,3), activation = 'relu'))
model.add(MaxPool2D(2,2))
model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))
model.summary()


Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_34 (Conv2D)          (None, 153, 153, 16)      160       
                                                                 
 max_pooling2d_34 (MaxPoolin  (None, 76, 76, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_35 (Conv2D)          (None, 74, 74, 32)        4640      
                                                                 
 max_pooling2d_35 (MaxPoolin  (None, 37, 37, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_36 (Conv2D)          (None, 35, 35, 64)        18496     
                                                                 
 max_pooling2d_36 (MaxPoolin  (None, 17, 17, 64)     

In [93]:
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [67]:
prediction = model.fit(X, Y, epochs = 5)

ValueError: Data cardinality is ambiguous:
  x sizes: 8001
  y sizes: 2001
Make sure all arrays contain the same number of samples.

In [34]:
accuracy = prediction.history['accuracy']
val_accuracy = prediction.history['val_accuracy']
epochs = range(len(accuracy))

pt.plot(epochs, accuracy, 'g', label = 'Training Accuracy')
pt.plot(epochs, val_accuracy)

TypeError: got an unexpected keyword argument 'orientation'

<Figure size 640x480 with 1 Axes>

In [37]:
model.save('pet_classification.model')

Cause: Unable to locate the source code of <function trace_model_call.<locals>._wrapped_model at 0x000001D1C1827D90>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function trace_model_call.<locals>._wrapped_model at 0x000001D1C1827D90>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function trace_model_call.<locals>._wrapped_model at 0x000001D1C1827D90>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


INFO:tensorflow:Assets written to: pet_classification.model\assets


INFO:tensorflow:Assets written to: pet_classification.model\assets


In [39]:
model = keras.models.load_model('pet_classification.model')

In [40]:
import cv2
import matplotlib.pyplot as pt

In [41]:
img_num = 2
while os.path.isfile(f'DataForTesting/{img_num}.jpg'):
    try:
        img = cv2.imread(f'DataForTesting/{img_num}.jpg')
        # re_img = cv2.resize(img, (155,155), interpolation = cv2.INTER_LINEAR)
        # re_img = np.invert(np.array([re_img]))
        pred = model.predict(img)
        pt.imshow(img[0], cmap = pt.cm.binary)
        pt.show()
        print(f"the animal is cat if label is 00 and dog if 00, label : {np.argmax(pred)}")
    except:
        print('error while opening file')
    finally:
        img_num += 1


error while opening file
error while opening file
error while opening file
error while opening file
error while opening file
error while opening file
error while opening file
